# Serial Position Cell Analysis

Analyze TH data for serial positon cells. 

In [ ]:
%config Completer.use_jedi = False

In [ ]:
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.stats import sem
import matplotlib.pyplot as plt

from pynwb import NWBHDF5IO

from spiketools.stats.shuffle import shuffle_spikes
from spiketools.stats.permutations import compute_surrogate_stats, compute_empirical_pvalue, zscore_to_surrogates
from spiketools.plts.space import plot_positions
from spiketools.plts.stats import plot_surrogates
from spiketools.plts.utils import make_axes
from spiketools.plts.data import plot_bar
from spiketools.plts.annotate import color_pval
from spiketools.utils import set_random_seed
from spiketools.utils.data import restrict_range, get_value_by_time, get_value_by_time_range

In [ ]:
import seaborn as sns
sns.set_context('talk')

In [ ]:
import sys
sys.path.append('../code')
from analysis import get_spike_positions
from serial import compute_serial_position_fr, create_df_serial, fit_anova_serial

# Settings

In [ ]:
# Set random seed, for consistency shuffling
set_random_seed(101)

In [ ]:
# Define the base data folder
data_folder = Path('/Users/Claire_han/Documents/Data/JacobsLab/TH/')
#data_folder = Path('/Users/tom/Documents/Data/JacobsLab/TH/')
# data_folder = Path('/scratch/tom.donoghue/TH/')

In [ ]:
# Define subject information
task = 'THF'
subj = 'wv001'
session = 2

In [ ]:
# Shuffle Settings
shuffle_approach = 'CIRCULAR'  # 'CIRCULAR', 'BINCIRC'
n_surrogates = 500

if shuffle_approach == 'CIRCULAR':
    shuffle_kwargs = {}
if shuffle_approach == 'BINCIRC':
    shuffle_kwargs = {}
    #shuffle_kwargs = {'shuffle_min' : 20000}

In [ ]:
# Set example unit to analyze
uind = 74

In [ ]:
# Set which analysis method to run
METHOD = 'ANOVA'  # 'SEGMENT' / 'ANOVA'

# Load Data

In [ ]:
# Define expected NWB file name & full path
file_name = '_'.join([task, subj, 'session_' + str(session)]) + '.nwb'
full_path = data_folder / 'NWB' / file_name

In [ ]:
# Load NWB file
io = NWBHDF5IO(str(full_path), 'r')
nwbfile = io.read()

#### Get Trial Data

In [ ]:
# Get the number of trials
n_trials = len(nwbfile.trials)
print('Number of trials: {}'.format(n_trials))

In [ ]:
# Extract behavioural markers of interest
nav_starts = nwbfile.trials['navigation_start'].data[:]
chest_openings = nwbfile.trials['chest_opening_time'][:]

#### Get Unit Data

In [ ]:
# Check the number of units & keep indices
n_units = len(nwbfile.units)
keep_inds = np.where(nwbfile.units.keep[:])[0]
n_keep = len(keep_inds)
print('Number of units: {}'.format(n_units))
print('Number to keep:  {}'.format(n_keep))

In [ ]:
# Get spikes
spikes = nwbfile.units.get_unit_spike_times(uind)

#### Get Position Data

In [ ]:
# Extract the position data
ptimes = nwbfile.acquisition['position']['player_position'].timestamps[:]
positions = nwbfile.acquisition['position']['player_position'].data[:].T

In [ ]:
# Get the chest positions & trial indices
chest_xs, chest_ys = nwbfile.acquisition['stimuli']['chest_positions'].data[:].T
chest_trials = nwbfile.trials.chest_trials[:]

# Single Trial

In [ ]:
# Get trial information
t_ind = 10
t_st = nwbfile.trials['navigation_start'][t_ind]
ch_openings = nwbfile.trials['chest_opening_time'][t_ind]
t_en = ch_openings[-1]

In [ ]:
# Get the chests for the current trial
t_mask = chest_trials == t_ind

In [ ]:
# Select chest openings for the current trial
t_time, t_pos = get_value_by_time_range(ptimes, positions, t_st, t_en)
ch_times = [get_value_by_time(t_time, t_pos, ch_op) for ch_op in ch_openings]

In [ ]:
# Restrict spikes to the selected trial
t_spikes = restrict_range(spikes, t_st, t_en)
t_spike_xs, t_spike_ys = get_spike_positions(t_spikes, t_time, t_pos)
spike_positions = np.array([t_spike_xs, t_spike_ys])

In [ ]:
# Plot an encoding trial: with traversals, chest locations, and spike firing
tspikes = {'positions' : np.array([t_spike_xs, t_spike_ys]), 'ms' : 10, 'alpha' : 0.6}
landmarks = [{'positions' : np.array([chest_xs[t_mask], chest_ys[t_mask]]),
              'color' : 'green', 'ms' : 50, 'alpha' : 0.25},
             {'positions' : np.atleast_2d(t_pos[:, 0]).T, 'color' : 'blue', 'ms' : 15},
             {'positions' : np.array(ch_times).T, 'color' : 'y', 'ms' : 15}]
plot_positions(t_pos, tspikes, landmarks, alpha=0.75, figsize=(5, 7))

# Serial Position

### Single Trial

In [ ]:
# Get the length of time between segments (chest openings)
seg_times = np.diff(np.insert(ch_openings, 0, t_st))

In [ ]:
# Count firing in each segment, and compute firing rate
count = Counter({0 : 0, 1 : 0, 2 : 0, 3 : 0})
count.update(np.digitize(t_spikes, ch_openings))
frs = np.array(list(count.values())) / seg_times

In [ ]:
# Plot firing rate per segment (single trial)
plot_bar(frs, count.keys())

### All Trials

In [ ]:
# Compute firing rates per segment for all trials
all_frs = compute_serial_position_fr(spikes, nav_starts, chest_openings, chest_trials)

In [ ]:
# Plot firing rates per segment across all trials
plot_bar(all_frs.mean(0), count.keys(), yerr=sem(all_frs, 0))

## Statistical Analyses

In [ ]:
# Create shuffled time series for comparison
times_shuffle = shuffle_spikes(spikes, shuffle_approach, n_surrogates, **shuffle_kwargs)

## Segment

In [ ]:
# compute the firing rate per segement across trials on surrogates
all_surrs = np.zeros(shape=[n_surrogates, 4])
for ind, stimes in enumerate(times_shuffle):
    surrs_frs = compute_serial_position_fr(\
        stimes, nav_starts, chest_openings, chest_trials) 
    all_surrs[ind] = np.mean(surrs_frs, 0)

In [ ]:
# Compute the empirical p-values and z-scores for each segment
p_vals = [compute_empirical_pvalue(all_frs.mean(0)[ind], all_surrs[:, ind]) for ind in range(4)]
z_scores = [zscore_to_surrogates(all_frs.mean(0)[ind], all_surrs[:, ind]) for ind in range(4)]

In [ ]:
# Plot distribution of surrogates with measured empirical value for each segment (seg1, seg2, seg3, seg4)
for ii, ax in zip(range(4), make_axes(4, wspace=0.4)):
    plot_surrogates(all_surrs[:,ii], all_frs.mean(0)[ii], ax=ax)
    title = 'Seg{}: z-{:1.2f}, p-{:1.2f}'.format(ii, z_scores[ii], p_vals[ii])
    ax.set_title(title, fontdict={'fontsize' : 15}, color=color_pval(p_vals[ii]));

In [ ]:
# Plot firing rates per segment across all trials, indicating shuffle significance
plot_bar(all_frs.mean(0), count.keys(), yerr=sem(all_frs, 0))
ax=plt.gca()
for ind, p_val in enumerate(p_vals):
    if p_val < 0.05:
        ax.text(ind, ax.get_ylim()[1]-0.15*ax.get_ylim()[1],
                '*', c='red', fontdict={'fontsize' : 25}, ha='center')

## ANOVA Analysis

In [ ]:
# Create dataframe
df = create_df_serial(all_frs)

In [ ]:
# Fit the ANOVA, and get f-value
f_val = fit_anova_serial(df)
f_val

In [ ]:
# Compute measures across shuffled time series for comparison
surrogates = np.zeros(n_surrogates)
for ind, shuffle in enumerate(times_shuffle):
    all_frs = compute_serial_position_fr(shuffle, nav_starts, chest_openings, 
                                         chest_trials)
    df = create_df_serial(all_frs)
    surrogates[ind] = fit_anova_serial(df)

In [ ]:
# Compute surrogate statistics
p_val, z_score = compute_surrogate_stats(f_val, surrogates)

# All Cells

In [ ]:
# # Plot the firing rate per segment across trials for all units
# p_val, z_score = '', ''
# for ii, ax in zip(keep_inds, make_axes(n_keep, hspace=0.4, wspace=0.4)):

#     spikes = nwbfile.units.get_unit_spike_times(ii)    
    
#     all_frs = compute_serial_position_fr(spikes, nav_starts, chest_openings,
#                                          chest_trials)

#     if METHOD == 'ANOVA':
#         data_value = fit_anova_serial(create_df_serial(all_frs))
    
#     # Shuffle analysis
#     if METHOD == 'SEGMENT':
#         surrogates = np.zeros(shape=[n_surrogates, 4])
#     if METHOD == 'ANOVA':
#         surrogates = np.zeros(n_surrogates)
#     times_shuffle = shuffle_spikes(spikes, shuffle_approach, n_surrogates, **shuffle_kwargs)
#     for ind, shuffle in enumerate(times_shuffle):

#         all_frs = compute_serial_position_fr(shuffle, nav_starts, chest_openings, 
#                                              chest_trials)
        
#         if METHOD == 'ANOVA':
#             surrogates[ind] = fit_anova_serial(create_df_serial(all_frs))
#         if METHOD == 'SEGMENT':
#             surrs_frs = compute_serial_position_fr(\
#                 shuffle, nav_starts, chest_openings, chest_trials) 
#             surrogates[ind] = np.mean(surrs_frs, 0)

#     if METHOD == 'SEGMENT':
#         p_vals = [compute_empirical_pvalue(all_frs.mean(0)[ind], surrogates[:, ind]) for ind in range(4)]
#         z_scores = [zscore_to_surrogates(all_frs.mean(0)[ind], surrogates[:, ind]) for ind in range(4)]
#         title = 'U{}'.format(ii)
#         color = 'black'
        
#     if METHOD == 'ANOVA':
#         p_val, z_score = compute_surrogate_stats(data_value, surrogates, plot=False, verbose=False)
#         title = 'U{} \n z-{:1.2f}, p-{:1.2f}'.format(ii, z_score, p_val)
#         color = color_pval(p_val)

#     plot_bar(all_frs.mean(0), count.keys(), yerr=sem(all_frs, 0), ax=ax)
#     ax.set_title(title, fontdict={'fontsize' : 12}, color=color)
    
#     if METHOD == 'SEGMENT':
#         for ind, p_val in enumerate(p_vals):
#             if p_val < 0.05:
#                 ax.text(ind, ax.get_ylim()[1]-0.15*ax.get_ylim()[1],
#                         '*', c='red', fontdict={'fontsize' : 25}, ha='center')